In [18]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# magic command to display matplotlib plots inline within the ipython notebook
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import catboost as cb

In [19]:
import urllib
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime

from collections import namedtuple
import pickle
from IPython.display import clear_output
from selenium import webdriver
from time import sleep

In [20]:
import re

In [21]:
def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

In [22]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [23]:
train['isTrain'] = True
test['isTrain'] = False

In [24]:
X = train.append(test, ignore_index=True)

In [25]:
X.isnull().sum()

address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat             420
long            420
target         2504
dtype: int64

# Геокодирование + заполнение пропусков c помощью геокодера Yandex

In [33]:
#https://geocode-maps.yandex.ru/1.x/?geocode=Moscow+Lva+Tolstogo+Street+16&lang=en_US
import requests
import json
YANDEX_URL = "https://geocode-maps.yandex.ru/1.x"
STATIC = "/?format=json&lang=en_US&geocode="
def geocode(address):
        response = requests.get(YANDEX_URL + STATIC + address)
        parsed_dict = json.loads(response.content)
        try:
            pos =  parsed_dict['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
            lat, lon = [float(item) for item in pos.split()]
            return lat, lon
        except:
            return None, None

In [34]:
X[['long','lat']] = X.apply(lambda row: geocode(row['address']) if (np.isnan(row['lat'])|np.isnan(row['long']))\
                                    else (row['long'],row['lat']), axis = 1).apply(pd.Series)

In [35]:
X.isnull().sum()

address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat             102
long            102
target         2504
street            0
city              0
new_address       0
dtype: int64

In [36]:
# остатки заполняла руками - 76 штук (надо попробовать selenium)

In [40]:
X[X.lat.isnull()].head()

,address,address_rus,atm_group,id,isTrain,lat,long,target,street,city,new_address
40,PER. BAZARNY 4 SPASSK-DALNY,NaN,496.5,4510.0,True,NaN,NaN,0.039191,PER. BAZARNY 4,SPASSK-DALNY,PER. BAZARNY 4SPASSK-DALNY
334,38 ULYANOVSKOE H/W SYZRAN,NaN,1942.0,1380.0,True,NaN,NaN,0.053041,38 ULYANOVSKOE H/W,SYZRAN,38 ULYANOVSKOE H/WSYZRAN
440,PRESNENSKAJA MOSKVA,NaN,3185.5,2491.0,True,NaN,NaN,0.006006,PRESNENSKAJA,MOSCOW,PRESNENSKAJAMOSCOW
475,"KR.G. ROSSIJSKAJA,6 OMSK",NaN,3185.5,2746.0,True,NaN,NaN,-0.122659,"KR. G. ROSSIJSKAJA, 6",OMSK,"KR. G. ROSSIJSKAJA, 6OMSK"
587,"D. 6, PR-KT UL'YANOVSKII ULYANOVSK G",NaN,5478.0,5169.0,True,NaN,NaN,-0.010810,"6, PROSPECT UL'YANOVSKI",I ULYANOVSK,"6, PROSPECT UL'YANOVSKII ULYANOVSK"


In [38]:
#X[X.lat.isnull()][:20].apply(lambda row: geocode(row['address']) if (np.isnan(row['lat'])|np.isnan(row['long'])) else (row['lat'],row['long']), axis = 1).apply(pd.Series)

In [86]:
row = 2259
lat, longg = 55.682336, 52.295119

In [87]:
X.at[row,'lat'] = lat
X.at[row,'long'] = longg

In [89]:
#X.to_csv('X_without_nulls_2.csv')

In [140]:
X.loc[214]

,address,address_rus,atm_group,id,isTrain,lat,long,target
214,133A MOZHAYSKOE H/W. MOSCOW,NaN,1942.0,556.0,True,46.729243,-116.997544,0.061517
214,"D. 65/1, PER. SOVETSKII BLAGOVESHCHE","Советский переулок, 65/1, Благовещенск, Амурск...",5478.0,5434.0,False,127.554329,50.267776,NaN


In [ ]:
#! python -m visdom.server

# Уточнение адресов с помощью selenium

In [39]:
driver = webdriver.Chrome('./chromedriver')

In [42]:
url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
#urli = url + str(X.address[2])
#urli

In [43]:
#driver.get(urli)
#element = driver.find_element_by_xpath("//div[@class='card-title-view']")

In [432]:
print element.text

Коммунистический проспект
Южно-Сахалинск, Сахалинская область, Россия
46.959413, 142.741113
Маршрут


In [44]:
i = 0
for index in X.index:
    print X.address[index], index
    urli = url + str(X.address[index])
    print urli
    driver.get(urli)
    try: 
        element = driver.find_element_by_xpath("//div[@class='card-title-view']")
        X.loc[i, 'info'] = element.text
        i+=1
    except:
        pass
    sleep(0.2)

EMELYANOVA,34          Y-SAKHALINSK    0
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=EMELYANOVA,34          Y-SAKHALINSK   
KOMSOMOLSKAYA,259B     Y.SAKHALINSK    1
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=KOMSOMOLSKAYA,259B     Y.SAKHALINSK   
KOMMUN. PR., 32        YUZHNO SAKHAL   2
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=KOMMUN. PR., 32        YUZHNO SAKHAL  
LENINGRADSKIY PR.,76A  MOSCOW          3
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=LENINGRADSKIY PR.,76A  MOSCOW         
GVARDEYSKAYA PL., 2    NORILSK         4
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=GVARDEYSKAYA PL., 2    NORILSK        
RUSSKAYA, 16           VLADIVOSTOK     5
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=RUSSKAYA, 16           VLADIVOSTOK    
SHUKHOVA, 14           MOSCO

KLUCHEVSKAYA 6B           ULAN-UDE     51
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=KLUCHEVSKAYA 6B           ULAN-UDE    
BONDARENKO 37             IVANOVKA     52
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=BONDARENKO 37             IVANOVKA    
SOVETSKAYA 29             CHERDAKLY    53
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=SOVETSKAYA 29             CHERDAKLY   
LENINA 7                  YASNOGORSK   54
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=LENINA 7                  YASNOGORSK  
POCHTOVIY PER, 3          ELIZOVO      55
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=POCHTOVIY PER, 3          ELIZOVO     
SHUHOVA 26                TULA         56
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=SHUHOVA 26                TULA        
PROLETARSKAYA 11      

AK. KIRENSKOGO 2          KRASNOYARSK  102
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=AK. KIRENSKOGO 2          KRASNOYARSK 
LENINA 23                 BRYANSK      103
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=LENINA 23                 BRYANSK     
KIROVA 7A                 ASTRAKHAN    104
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=KIROVA 7A                 ASTRAKHAN   
OKTYABRSKAYA 4            VOLODARSKIY  105
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=OKTYABRSKAYA 4            VOLODARSKIY 
NABEREZHNAYA 1 MAY, 160   ASTRAKHAN    106
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=NABEREZHNAYA 1 MAY, 160   ASTRAKHAN   
BEZHITSKAYA 1/5           BRYANSK      107
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=BEZHITSKAYA 1/5           BRYANSK     
KRILATAYA 2     

ABB 50A POBEDY PR.        KAZAN        152
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=ABB 50A POBEDY PR.        KAZAN       
ABB 77 ZORGE              KAZAN        153
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=ABB 77 ZORGE              KAZAN       
ABB 65 SHAMILYA USMANOVA  NAB.CHELNY   154
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=ABB 65 SHAMILYA USMANOVA  NAB.CHELNY  
ABB 168 B. KRASNAYA       VYS.GORA,    155
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=ABB 168 B. KRASNAYA       VYS.GORA,   
ABB 32 LENINGRADSKAYA     KAZAN        156
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=ABB 32 LENINGRADSKAYA     KAZAN       
ABB 113A LENINA           ALMETEVSK    157
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=ABB 113A LENINA           ALMETEVSK   
ABB 32 LENINA   

138 MICHURINA STR         SAMARA       202
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=138 MICHURINA STR         SAMARA      
46/1 VYSOTSKOGO STR       NOVOSIBIRSK  203
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=46/1 VYSOTSKOGO STR       NOVOSIBIRSK 
OTKRITOE SHOSSE 9         MOSCOW       204
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=OTKRITOE SHOSSE 9         MOSCOW      
21B LENINA AV.            YAROSLAVL    205
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=21B LENINA AV.            YAROSLAVL   
89 MELNIKAYTE STR         TYUMEN       206
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=89 MELNIKAYTE STR         TYUMEN      
32 NOVOKOSINSKAYA         MOSCOW       207
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=32 NOVOKOSINSKAYA         MOSCOW      
20 SHEREMETEVSKA

...

D. 6, UL. KOMSOMOL'SKAYA  YAROSLAVL G  8552
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 6, UL. KOMSOMOL'SKAYA  YAROSLAVL G 
VORONOVA 14G              KRASNOJARSK  8553
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=VORONOVA 14G              KRASNOJARSK 
SCHADENKO 70              KAMENSK-SHAH 8554
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=SCHADENKO 70              KAMENSK-SHAH
CHEKALINA, 15          Novosibirsk     8555
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=CHEKALINA, 15          Novosibirsk    
D. 60, UL. SOVETSKAYA     TAMBOV G     8556
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 60, UL. SOVETSKAYA     TAMBOV G    
SUDOSTROITELN 58          KRASNOJARSK  8557
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=SUDOSTROITELN 58          KRASNOJARSK 
KRAS. RAB.

Petra Alekseyeva, 83a  Yakutsk         8602
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=Petra Alekseyeva, 83a  Yakutsk        
SOVETSKAYA 8              BELOVO       8603
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=SOVETSKAYA 8              BELOVO      
D. 10, UL. KARLA MARKSA   KURSK G      8604
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 10, UL. KARLA MARKSA   KURSK G     
MIKHALEVICHA, 1        Ramenskoye      8605
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=MIKHALEVICHA, 1        Ramenskoye     
12 KASHIRSKOE H/W.        MOSCOW       8606
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=12 KASHIRSKOE H/W.        MOSCOW      
MOROZOVA P.L., 86      KHabarovsk      8607
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=MOROZOVA P.L., 86      KHabarovsk     
31-1 M.ZUK

D. 30, UL. ZHELEZNODOROZH MGA PGT      8652
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 30, UL. ZHELEZNODOROZH MGA PGT     
65 KOSMONAVTOV H/W        PERM         8653
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=65 KOSMONAVTOV H/W        PERM        
27/39 TIPANOVA STR.       S.-PETERSBUR 8654
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=27/39 TIPANOVA STR.       S.-PETERSBUR
D. 44/46, UL. SARATOVSKAY KRASNYY KUT  8655
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 44/46, UL. SARATOVSKAY KRASNYY KUT 
153 LYUBLINSKAYA ST       MOSCOW       8656
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=153 LYUBLINSKAYA ST       MOSCOW      
D. 26G, UL. ORLOVSKAYA    MUROM G      8657
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 26G, UL. ORLOVSKAYA    MUROM G     
MOSKOVSKAY

D. 23, PR-KT AKSAISKII    AXAY G       8702
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 23, PR-KT AKSAISKII    AXAY G      
EMELYANOVA, 32A        YUzhno-Sakhalin 8703
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=EMELYANOVA, 32A        YUzhno-Sakhalin
AKADEMIKA PLATONOVA 10    SAMARA       8704
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=AKADEMIKA PLATONOVA 10    SAMARA      
21-2 B. ORDYNKA STR.      MOSCOW       8705
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=21-2 B. ORDYNKA STR.      MOSCOW      
Krasnyj Put', 153/2       OMSK         8706
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=Krasnyj Put', 153/2       OMSK        
SOVETSKAYA 25             VAVOZH       8707
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=SOVETSKAYA 25             VAVOZH      
D. 23 A, U

D. 26A, UL. PERVOMAISKAYA BERDSK G     8752
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=D. 26A, UL. PERVOMAISKAYA BERDSK G    
VL.4 NOSOVIKHINSKOE HW.   MKR.SALTYKOV 8753
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=VL.4 NOSOVIKHINSKOE HW.   MKR.SALTYKOV
54A 50-LET OKTYABRY       SYZRAN       8754
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=54A 50-LET OKTYABRY       SYZRAN      
MIRA 231                  YUZHNO-SAKHA 8755
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=MIRA 231                  YUZHNO-SAKHA
KARLA MARKSA 36A          ERMAKOVSKOE  8756
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=KARLA MARKSA 36A          ERMAKOVSKOE 
OKTYABRESKAY, 108      Volsk           8757
https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text=OKTYABRESKAY, 108      Volsk          
EGERSKIY B

In [51]:
X.isnull().sum()

address           0
address_rus     420
atm_group         0
id                0
isTrain           0
lat             102
long            102
target         2504
street            0
city              0
new_address       0
info           6149
dtype: int64

In [86]:
#X.to_csv('X_with_info.csv')

In [ ]:
#def parsing(X, column='info', start=0):
#    driver = webdriver.Chrome('./chromedriver')
#    X[column] = np.nan
#    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
#    for address, city in zip(X.address, X.city):
#        print address, city
#        urli = url + ' '.join([address,city]).replace(' ', '%20')
##        driver.get(urli)
#      try: 
#            element = driver.find_element_by_xpath("//div[@class='card-title-view']")
#            X.loc[i, column] = element.text
#        except:
#            pass
#        sleep(0.2)
#    return X.copy()

# Геокодирование + заполнение пропусков для отделений Росбанка

In [107]:
#https://geocode-maps.yandex.ru/1.x/?geocode=Moscow+Lva+Tolstogo+Street+16&lang=en_US
import requests
import json
import pandas as pd
YANDEX_URL = "https://geocode-maps.yandex.ru/1.x"
STATIC = "/?format=json&lang=en_US&geocode="
def geocode(address):
        response = requests.get(YANDEX_URL + STATIC + address)
        parsed_dict = json.loads(response.content)
        try:
            pos =  parsed_dict['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']
            lat, lon = [float(item) for item in pos.split()]
            return lat, lon
        except:
            return None, None

In [82]:
departments = pd.read_csv('rosbank_departments.csv')

In [83]:
departments.head()

,Unnamed: 0,Address
0,19,"Московская область, Серпухов, 1-я Московская у..."
1,0,"Московская область, Дубна, проспект Боголюбова..."
2,1,"Республика Башкортостан, Салават, Октябрьская ..."
3,2,"Архангельская область, Новодвинск, улица Ворош..."
4,3,"Алтайский край, Славгород, улица Володарского,..."


In [85]:
departments.shape

(404, 2)

In [111]:
departments[['long','lat']] = departments.Address.apply(lambda x: geocode(x)).apply(pd.Series)

In [113]:
departments['is_department'] = 1

In [115]:
departments.to_csv('departments_with_lat.csv')

# Геокодирование для отделений Сбербанка

In [70]:
sber_departments = pd.read_csv('sber_departments.csv')

In [71]:
print sber_departments.address[1]

г.Белгород улица Некрасова 5/8, корп.Б


In [72]:
sber_departments.head()

,Unnamed: 0,address
0,0,г.Белгород улица Князя Трубецкого 24
1,1,"г.Белгород улица Некрасова 5/8, корп.Б"
2,2,г.Алексеевка переулок 1-й К. Маркса 2
3,10,г.Алексеевка улица Комсомольская 46А
4,17,г.Белгород улица Железнякова 18-А


In [73]:
sber_departments[:3].address.apply(lambda x: geocode(x)).apply(pd.Series)

,0,1
0,36.596197,50.592681
1,36.582947,50.614560
2,38.691257,50.629458


In [74]:
sber_departments[['long','lat']] = sber_departments.address.apply(lambda x: geocode(x)).apply(pd.Series)

In [63]:
geocode('Белгородская областьг.Белгород улица Некрасова 5/8, корп.Б')

(38.379029, 50.658503)

In [78]:
sber_departments['is_sber'] = 1

In [80]:
sber_departments.to_csv('sber_departments.csv')

In [88]:
sber_departments.head()

,Unnamed: 0,address,long,lat,is_sber
0,0,г.Белгород улица Князя Трубецкого 24,36.596197,50.592681,1
1,1,"г.Белгород улица Некрасова 5/8, корп.Б",36.582947,50.614560,1
2,2,г.Алексеевка переулок 1-й К. Маркса 2,38.691257,50.629458,1
3,10,г.Алексеевка улица Комсомольская 46А,70.963126,51.981829,1
4,17,г.Белгород улица Железнякова 18-А,36.576209,50.632520,1
